In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
df = yf.download ('NVDA')

[*********************100%***********************]  1 of 1 completed


In [4]:
df['SMA200'] = df. Close.rolling (200).mean()

In [5]:
df['SMA5'] = df. Close.rolling (5).mean()

In [6]:
df ['10d_low'] = df. Close.rolling (10).min()

In [25]:
df ['Buy'] = np.where((df.Close > df.SMA200)&
                     (df ['10d_low'].diff() < 0) &
                     (0.98* df.Close > df. Low.shift(-1)), 1, 0)

In [26]:
df ['Sell'] = np.where(df.Close > df.SMA5, 1, 0)

In [27]:
df ['Buyprice'] = 0.98* df.Close

In [28]:
df ['Sellprice'] = df.Open.shift (-1)

In [29]:
df

,Open,High,Low,Close,Adj Close,Volume,SMA200,SMA5,10d_low,Sell,Buyprice,Sellprice,Buy
Date,,,,,,,,,,,,,
1999-01-22,0.437500,0.488281,0.388021,0.410156,0.376820,271468800.0,NaN,NaN,NaN,0,0.401953,0.442708,0
1999-01-25,0.442708,0.458333,0.410156,0.453125,0.416296,51048000.0,NaN,NaN,NaN,0,0.444062,0.458333,0
1999-01-26,0.458333,0.467448,0.411458,0.417969,0.383998,34320000.0,NaN,NaN,NaN,0,0.409610,0.419271,0
1999-01-27,0.419271,0.429688,0.395833,0.416667,0.382801,24436800.0,NaN,NaN,NaN,0,0.408334,0.416667,0
1999-01-28,0.416667,0.419271,0.412760,0.415365,0.381605,22752000.0,NaN,0.422656,NaN,0,0.407058,0.415365,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-03,266.700012,267.839996,262.350006,265.980011,265.980011,23991000.0,177.144550,258.668002,226.919998,1,260.660411,272.290009,0
2021-11-04,272.290009,313.649994,271.179993,298.010010,298.010010,115363100.0,177.941225,268.388004,227.259995,1,292.049810,301.869995,0
2021-11-05,301.869995,314.000000,294.100006,297.519989,297.519989,85072800.0,178.743200,276.758002,231.660004,1,291.569589,301.489990,0


In [30]:
x = df [(df.Buy == 1) | (df.Sell == 1)]

In [35]:
y = x[(x.Buy.diff() == 1) | (x.Sell.diff() == 1)]

In [36]:
profits = ((y.Sellprice. shift (-1) - y.Buyprice)/y.Buyprice)

In [39]:
profits = profits[::2]

In [45]:
def backtest(arr):
    winrate = len(arr [arr > 0])/len(arr)
    max_dd = min(arr)
    mean = arr.mean()
    gain = (arr + 1).cumprod () [-1] 
    return winrate, max_dd, mean, gain

In [46]:
backtest(profits)

(0.64, -0.26571127808228173, 0.15595186550496845, 419.6566356259198)

In [54]:
def manipulatedf (df):
    df = df.copy()
    df ['SMA200'] = df.Close.rolling(200).mean()
    df ['SMA5'] = df. Close.rolling (5).mean()
    df ['10d_low'] = df.Close.rolling (10).min()
    df.dropna (inplace=True)
    df ['Buy'] = np.where((df.Close > df.SMA200) &
                     (df['10d_low'].diff() < 0) &
                     (0.98* df.Close >= df. Low.shift (-1)),1,0)
    df ['Sell'] = np.where(df.Close > df.SMA5, 1, 0)
    df ['Buyprice'] = 0.98 * df.Close
    df ['Sellprice'] = df.Open. shift (-1)
    x = df [(df.Buy == 1) | (df.Sell == 1)]
    y = x[(x.Buy.diff() == 1) | (x.Sell.diff() == 1)]
    profits = ((y.Sellprice. shift (-1)- y.Buyprice)/y.Buyprice)
    profits = profits[::2]
    return profits

In [55]:
backtest (manipulatedf(df))

(0.63, -0.5158176397469406, 0.008413849135172246, 1.5363852147475627)

In [65]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

In [66]:
tickers = tickers.Symbol.to_list()

In [70]:
profitmatrix = []

for ticker in tickers:
    df = yf.download (ticker, start='2000-01-01')
    prof = manipulatedf(df) 
    profitmatrix.append(prof)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [71]:
profitmatrix = [i for i in profitmatrix if len(i) > 0]

In [72]:
results = []

for ele in profitmatrix: results.append (backtest (ele))

In [73]:
frame = pd.DataFrame (results)

In [74]:
frame.columns= ['winrate', 'max_dd', 'mean', 'gain']

In [75]:
frame.mean()

winrate    0.667440
max_dd    -0.154521
mean       0.010122
gain       1.443429
dtype: float64